기능별 collection 생성

In [ ]:
import os
import json
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import chromadb

#API Key 설정 
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

#임베딩 모델 초기화 
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#json-> text 파일을 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

# Chroma 클라이언트
client = chromadb.PersistentClient(path="./chroma_db")

#json 파일 정리
def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for item in data: #얘는 파일 별 수정해야해요요
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={"세부전공": item.get("세부전공", ""), "카테고리": item.get("카테고리", "")}
                ))
    return docs

#결과 확인 (Move this after embeddings are defined)

#json -> documents 파일 변환
def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for item in data:
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={"세부전공": item.get("세부전공", ""), "카테고리": item.get("카테고리", "")}
                ))
    return docs

#각 기능별 json 경로 리스트 
task_datasets = {
    "lecture_search": ["./data/강의탐색1.json", "./data/강의탐색2.json"],
    "career_counsel": ["./data/진로상담1.json", "./data/진로상담2.json"],
    "academic_status": ["./data/학업현황1.json", "./data/학업현황2.json"]
}

# 각 기능별 컬렉션 생성 및 임베딩 처리
for task, files in task_datasets.items():
    # JSON → Documents
    docs = load_json_documents(files)

    #청크 분할
    split_docs = text_splitter.split_documents(docs)

    # 청크 텍스트
    texts = [doc.page_content for doc in split_docs]

    # 임베딩
    embeddings = embeddings_model.embed_documents(texts)

    # 저장
    collection = client.get_or_create_collection(name=task)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{task}_{i}" for i in range(len(texts))]
    )
    
    print("임베딩 개수:", len(embeddings))
    print("벡터 차원:", len(embeddings[0]))

사용자의 입력을 기능별로 분류
강의 탐색 -> 강의탐색기능 
학습 현황 -> 학습현황기능
진로 상담 -> 진로상담기능 

In [ ]:
def classify_function(user_input: str) -> str:
    if "강의" in user_input or "탐색" in user_input:
        return "lecture_search"
    elif "진로" in user_input or "상담" in user_input:
        return "career_counsel"
    elif "학업" in user_input or "현황" in user_input:
        return "academic_status"
    else:
        return "lecture_search"  # fallback

분류된 기능에 맞춰 ChromaDB 검색

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

def answer_query(user_input: str):
    task = classify_function(user_input)

    # 컬렉션 로드
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=task,
        embedding_function=embeddings_model
    )

    # 유사 문서 검색
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

    # 답변 생성
    return qa.run(user_input)